# Jan 28, 2026: list all graphs

conda env: gt

In [1]:
import os
import sys
import numpy as np 
import pandas as pd 
from glob import glob
from nilearn.maskers import NiftiLabelsMasker
import nibabel as nib
import re
from pathlib import Path
import dill as pickle
from tqdm import tqdm
import graph_tool.all as gt 
from scipy.stats import entropy, zscore
from joblib import Parallel, delayed
from itertools import product, combinations
from sklearn.covariance import GraphicalLasso

# plotting
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cycler import cycler
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

# import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS(): pass

args = ARGS()

args.SEED = 100
np.random.seed(args.SEED)

In [3]:
PARC_DESC = f'NEWMAX_ROIs_final_gm_100_2mm' #f'NEWMAX_ROIs_final_gm_104_2mm' #f'ABA_ROIs_final_gm_36'
BASE_path = f'/home/govindas/lab-data/aba'

args.ANALYSIS = 'trial-end'
args.WINDOW = np.arange(5, 9+1)

ROI_path = (
    f'{BASE_path}/{PARC_DESC}/analysis-{args.ANALYSIS}'
)
os.makedirs(ROI_path, exist_ok=True)

TS_file = f'{BASE_path}/{PARC_DESC}/roi_timeseries_df.pkl'

In [4]:
GRAPH_DEFS = [f'constructed']
GRAPH_METHODS = [f'pearson'] # [f'pearson', f'mutualinfo']
THRESHOLDINGS = [f'signed', f'unsigned']
EDGE_DEFS = [f'binary', f'weighted']
EDGE_DENSITIES = [10, 20, 30] #[10, 15, 20, 25]
LAYER_DEFS = [f'individual'] #, f'multilayer']
DATA_UNITS = [f'grp', f'sub']

In [5]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'sub' #'grp'

In [6]:
GRAPH_FOLDERS = sorted(glob(f'{ROI_path}/graph-constructed/*/*/*/*/*/*/*', recursive=True))
GRAPH_FOLDERS

['/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-10/layer-individual/unit-sub/cond-highR',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-10/layer-individual/unit-sub/cond-highT',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-10/layer-individual/unit-sub/cond-lowR',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-10/layer-individual/unit-sub/cond-lowT',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-highR',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_1

In [7]:
for FOLDER in GRAPH_FOLDERS:
    files = sorted(glob(f'{FOLDER}/graphs/*', recursive=True))
    
    with open(f'{FOLDER}/all_graphs.txt', 'w') as f:
        f.writelines('\n'.join(files))